# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235470259666                   -0.50    8.0         
  2   -7.250349742129       -1.83       -1.40    1.0   11.1ms
  3   -7.251147567134       -3.10       -2.17    1.0   11.8ms
  4   -7.251235312278       -4.06       -2.11    3.0   16.0ms
  5   -7.251332697614       -4.01       -2.65    1.0   10.4ms
  6   -7.251334377974       -5.77       -2.82    2.0   14.0ms
  7   -7.251338565396       -5.38       -3.39    1.0   10.2ms
  8   -7.251338770182       -6.69       -3.86    2.0   12.4ms
  9   -7.251338795433       -7.60       -4.35    1.0   10.0ms
 10   -7.251338798346       -8.54       -4.74    2.0   11.7ms
 11   -7.251338798645       -9.52       -5.19    1.0   10.4ms
 12   -7.251338798697      -10.28       -5.64    2.0   12.8ms
 13   -7.251338798703      -11.20       -6.17    2.0   13.5ms
 14   -7.251338798704      -12.01       -6.36    2.0   12.6ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05401862292880719
[ Info: Arnoldi iteration step 2: normres = 0.5674546545279696
[ Info: Arnoldi iteration step 3: normres = 0.9208174388384764
[ Info: Arnoldi iteration step 4: normres = 0.3090135038420841
[ Info: Arnoldi iteration step 5: normres = 0.46825165700818583
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.91e-02, 7.40e-02, 4.09e-01, 2.11e-01, 1.36e-02)
[ Info: Arnoldi iteration step 6: normres = 0.30516212552190713
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.00e-02, 1.18e-01, 2.37e-01, 1.20e-01, 5.06e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07860153457694552
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.79e-04, 9.58e-03, 1.53e-02, 4.02e-02, 5.68e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09713636225200033
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.57e